# Domain Adaptation

In [19]:
!hf auth login

^C


In [ ]:
!pip install --upgrade transformers datasets accelerate peft bitsandbytes

In [ ]:
import transformers
import datasets
import accelerate
import peft
import bitsandbytes

print("All packages imported successfully!")
print(f"transformers: {transformers.__version__}")
print(f"datasets: {datasets.__version__}")
print(f"accelerate: {accelerate.__version__}")
print(f"peft: {peft.__version__}")
print(f"bitsandbytes: {bitsandbytes.__version__}")

All packages imported successfully!
transformers: 4.56.1
datasets: 4.0.0
accelerate: 1.10.1
peft: 0.17.1
bitsandbytes: 0.47.0


In [ ]:
from datasets import load_dataset

# Load all data once
dataset = load_dataset("json", data_files="../constraints/bedroom_lora_dataset.jsonl", split="train")

# Split automatically into train/validation
dataset = dataset.train_test_split(test_size=0.2, seed=42)

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 400
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 100
    })
})


In [ ]:
from transformers import AutoTokenizer

model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad token (LLaMA normally has none)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def format_example(batch):
    texts = [f"{p}\n{c}" for p, c in zip(batch["prompt"], batch["completion"])]
    return tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length"
    )

tokenized_dataset = dataset.map(
    format_example,
    batched=True,
    remove_columns=["prompt", "completion"]
)


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

model_id = "meta-llama/Llama-3.2-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,   # saves VRAM, needs bitsandbytes
    device_map="auto"
)

# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # LLaMA attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00002.safetensors:  16%|#6        | 807M/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:  34%|###3      | 493M/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./llama3.2-3b-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    bf16=False,
    push_to_hub=False
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


C:\Users\Admin\AppData\Local\Temp\ipykernel_15132\3030834965.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


Step,Training Loss
10,1.500200
20,0.358000
30,0.199100
40,0.189200
50,0.186700
60,0.182900
70,0.181500
80,0.176600
90,0.177600
100,0.177500


TrainOutput(global_step=150, training_loss=0.2806020704905192, metrics={'train_runtime': 183.7976, 'train_samples_per_second': 6.529, 'train_steps_per_second': 0.816, 'total_flos': 1.04079418195968e+16, 'train_loss': 0.2806020704905192, 'epoch': 3.0})

In [ ]:
model.save_pretrained("llama3.2-3b-furniture-adapted")
tokenizer.save_pretrained("llama3.2-3b-furniture-adapted")


('llama3.2-3b-furniture-adapted\\tokenizer_config.json',
 'llama3.2-3b-furniture-adapted\\special_tokens_map.json',
 'llama3.2-3b-furniture-adapted\\chat_template.jinja',
 'llama3.2-3b-furniture-adapted\\tokenizer.json')

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="llama3.2-3b-furniture-adapted",
    tokenizer=tokenizer,
    device_map="auto"
)

prompt = "Arrange furniture in a bedroom for elderly comfort.\nRoom size: 10x12 ft. Furniture: queen bed, wardrobe, chair. Constraints: 110 cm clear paths."
print(pipe(prompt, max_new_tokens=150, do_sample=True)[0]["generated_text"])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Arrange furniture in a bedroom for elderly comfort.
Room size: 10x12 ft. Furniture: queen bed, wardrobe, chair. Constraints: 110 cm clear paths. Furniture dimensions: queen bed (180x200 cm), wardrobe (120x60 cm), chair (100x100 cm).
Clear paths: 110 cm. Furniture dimensions. Constraints.
Place queen bed along north wall. Wardrobe along south/east/west wall with clear path of 110cm. Chair along south/east/west wall with clear path of 110cm. Clear paths along north/south/east/west walls.
Place queen bed along north wall. Wardrobe along south/east/west wall with clear path of 110cm. Chair along south/east/west wall with clear path of 110cm. Clear paths along north/south/east/west walls.
Place queen bed along north wall. Wardrobe
